In [180]:
import pandas as pd

In [181]:
vecs_df = pd.read_csv("../1bit_200d_nonbin", skiprows=[0], header=None, sep=" ")
vecs_df = vecs_df.set_index(0)
vecs_df = vecs_df.drop(columns=[201])
vecs_df = vecs_df.replace(vecs_df.values.max(), 1)
vecs_df = vecs_df.replace(vecs_df.values.min(), 0)
vecs_df.head()

,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
0,,,,,,,,,,,,,,,,,,,,,
</s>,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
the,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
of,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
and,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
one,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0


In [182]:
vecs_df.values.max()

1.0

In [183]:
len(vecs_df)

60238

In [184]:
## Exploratory analysis

In [185]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import WordNetError

In [186]:
### Top-level correlation for the groups of the same root-hypernym

In [187]:
def root_hypernym_for_word(n):
#     print("%s.n.01" % n)
    try:
        return wn.synsets(str(n))[0].root_hypernyms()[0].name()
    except IndexError:
        # not a noun
        return None

In [188]:
def all_hypernyms_for_word(n):
    try:
        return set([syn.root_hypernyms()[0] for syn in wn.synsets(str(n))])
    except IndexError:
        # not a noun
        return None

In [189]:
root_hypernym_for_word("was")

'entity.n.01'

In [190]:
vecs_df["root_hypernym"] = [root_hypernym_for_word(w) for w in list(vecs_df.index)]
vecs_df["root_hypernym"]

0
</s>                        None
the                         None
of                          None
and                         None
one                  entity.n.01
in                   entity.n.01
a                    entity.n.01
to                          None
zero                 entity.n.01
nine                 entity.n.01
is                       be.v.01
two                  entity.n.01
as                   entity.n.01
for                         None
eight                entity.n.01
s                    entity.n.01
by                       by.r.01
that                        None
was                  entity.n.01
five                 entity.n.01
three                entity.n.01
four                 entity.n.01
with                        None
six                  entity.n.01
seven                entity.n.01
on                       on.a.01
are                  entity.n.01
it                   entity.n.01
from                        None
or                   entity.n.01
        

In [191]:
[all_hypernyms_for_word(w) for w in list(vecs_df.index)]

[set(),
 set(),
 set(),
 set(),
 {Synset('entity.n.01'),
  Synset('matchless.s.01'),
  Synset('one.s.01'),
  Synset('one.s.02'),
  Synset('one.s.03'),
  Synset('one.s.04'),
  Synset('one.s.05'),
  Synset('one.s.06')},
 {Synset('entity.n.01'),
  Synset('in.r.01'),
  Synset('in.s.01'),
  Synset('in.s.02'),
  Synset('in.s.03')},
 {Synset('entity.n.01')},
 set(),
 {Synset('change.v.01'),
  Synset('entity.n.01'),
  Synset('zero.a.04'),
  Synset('zero.s.01'),
  Synset('zero.s.02'),
  Synset('zero.s.03')},
 {Synset('entity.n.01'), Synset('nine.s.01')},
 {Synset('act.v.01'),
  Synset('be.v.01'),
  Synset('be.v.02'),
  Synset('be.v.03'),
  Synset('be.v.05'),
  Synset('be.v.08'),
  Synset('be.v.11'),
  Synset('constitute.v.01'),
  Synset('equal.v.01'),
  Synset('exist.v.01'),
  Synset('use.v.03')},
 {Synset('entity.n.01'), Synset('two.s.01')},
 {Synset('entity.n.01'), Synset('equally.r.01')},
 set(),
 {Synset('eight.s.01'), Synset('entity.n.01')},
 {Synset('entity.n.01')},
 {Synset('aside.r.06')

In [206]:
# relevant root hypernyms
root_counts = vecs_df.groupby("root_hypernym").count()[1]
relevant_roots = root_counts[root_counts > root_counts.quantile(0.997)]
relevant_roots

root_hypernym
act.v.01           709
be.v.01            146
change.v.01        662
change.v.02        480
connect.v.01        88
entity.n.01      28870
express.v.02       172
get.v.01            91
make.v.03          386
move.v.02          377
move.v.03           97
think.v.03         386
transfer.v.05      162
travel.v.01        199
Name: 1, dtype: int64

In [193]:
### Correlations inside the groups

In [194]:
vecs_df.head(20)

,1,2,3,4,5,6,7,8,9,10,...,192,193,194,195,196,197,198,199,200,root_hypernym
0,,,,,,,,,,,,,,,,,,,,,
</s>,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,None
the,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,None
of,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,None
and,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,None
one,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,entity.n.01
in,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,entity.n.01
a,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,entity.n.01
to,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,None
zero,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,entity.n.01


In [219]:
# groups means of each vector attribute shows whether the difference between groups on given attribute is important
vecs_df.groupby("root_hypernym").mean().loc[relevant_roots.index][57]

root_hypernym
act.v.01         0.317348
be.v.01          0.458904
change.v.01      0.574018
change.v.02      0.633333
connect.v.01     0.704545
entity.n.01      0.569553
express.v.02     0.209302
get.v.01         0.428571
make.v.03        0.432642
move.v.02        0.596817
move.v.03        0.731959
think.v.03       0.373057
transfer.v.05    0.586420
travel.v.01      0.638191
Name: 57, dtype: float64

In [216]:
pd.set_option('display.max_columns', 50)  
vecs_df.groupby("root_hypernym").mean().loc[relevant_roots.index].describe()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,...,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200
count,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,...,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000
mean,0.421738,0.442091,0.568302,0.630187,0.432250,0.384548,0.495534,0.450348,0.482864,0.587739,0.564454,0.579640,0.418104,0.412687,0.573080,0.526260,0.596562,0.500852,0.362352,0.454033,0.497952,0.547831,0.553992,0.513657,0.494839,...,0.403126,0.471382,0.636645,0.471633,0.475945,0.505458,0.513761,0.546998,0.517687,0.502883,0.542673,0.488025,0.502590,0.560327,0.505287,0.442917,0.473516,0.457899,0.488825,0.462277,0.561669,0.435085,0.544041,0.478807,0.434958
std,0.061217,0.085409,0.062270,0.070533,0.068326,0.089592,0.081798,0.104471,0.109737,0.054239,0.110790,0.077201,0.094383,0.048511,0.034999,0.103518,0.098694,0.057574,0.057715,0.092290,0.057978,0.078813,0.071001,0.067210,0.090440,...,0.119015,0.061235,0.093431,0.045553,0.085675,0.046261,0.094450,0.070396,0.052403,0.065793,0.086033,0.093807,0.139892,0.077830,0.081050,0.100475,0.070855,0.077037,0.066741,0.071062,0.078528,0.087861,0.062768,0.057494,0.090567
min,0.329016,0.318182,0.439560,0.546392,0.271605,0.226744,0.348837,0.238372,0.284884,0.483516,0.401163,0.472603,0.281407,0.342466,0.515102,0.296703,0.417808,0.420455,0.271357,0.343023,0.373626,0.384615,0.428571,0.407407,0.358025,...,0.180233,0.381443,0.486301,0.376543,0.303109,0.384615,0.302326,0.397727,0.418605,0.418605,0.431818,0.314815,0.244186,0.431818,0.307692,0.295455,0.331395,0.351648,0.373626,0.318182,0.402010,0.284091,0.464150,0.352273,0.298969
25%,0.371532,0.386735,0.539286,0.580207,0.399389,0.337852,0.449289,0.398966,0.404207,0.553554,0.482913,0.524186,0.343313,0.370959,0.552749,0.473256,0.519498,0.443557,0.314942,0.381281,0.474191,0.512142,0.498161,0.461715,0.428929,...,0.337031,0.432672,0.560653,0.445823,0.441506,0.497347,0.465325,0.515484,0.484414,0.456166,0.475630,0.430253,0.384088,0.534211,0.459874,0.379203,0.454196,0.395010,0.447283,0.428981,0.506849,0.359799,0.505631,0.449417,0.371802
50%,0.430353,0.424087,0.561684,0.620310,0.444040,0.390224,0.508716,0.463915,0.524160,0.593772,0.569155,0.575848,0.415293,0.424331,0.567325,0.536279,0.603029,0.509922,0.371438,0.440614,0.500509,0.556425,0.550078,0.508278,0.471473,...,0.405420,0.470217,0.648578,0.481971,0.503769,0.507194,0.496093,0.535847,0.521681,0.489130,0.546836,0.499029,0.528465,0.552009,0.512166,0.427949,0.469350,0.447124,0.497409,0.460459,0.558307,0.447513,0.520063,0.475126,0.432152
75%,0.474939,0.505707,0.580072,0.664450,0.478686,0.424972,0.544409,0.520431,0.565110,0.619141,0.623922,0.620403,0.469979,0.432572,0.597451,0.614410,0.665434,0.541788,0.387600,0.493699,0.529584,0.597045,0.612103,0.551593,0.551773,...,0.459015,0.498173,0.702844,0.503365,0.522639,0.534660,0.576935,0.569550,0.557635,0.543706,0.581606,0.520353,0.621556,0.585738,0.569000,0.495378,0.516908,0.532836,0.518476,0.500115,0.630829,0.477546,0.572592,0.522248,0.502591
max,0.527397,0.598837,0.691860,0.796512,0.531250,0.556818,0.623457,0.615385,0.641975,0.668394,0.742268,0.715909,0.615385,0.512678,0.636364,0.674419,0.750000,0.601036,0.475309,0.639175,0.595890,0.659091,0.659091,0.636364,0.656977,...,0.602273,0.630137,0.750000,0.546392,0.583333,0.568182,0.659794,0.714286,0.607427,0.648352,0.728395,0.670455,0.701031,0.769231,0.608040,0.668605,0.623116,0.560062,0.587209,0.593407,0.662791,0.587629,0.660494,0.554795,0.598837


In [218]:
# attribute with max std:
vecs_df.groupby("root_hypernym").mean().loc[relevant_roots.index].std().idxmax()

57

In [211]:
# find where the extreme of attr_6 occurs and how stable it is
vecs_df.groupby("root_hypernym").mean().loc[relevant_roots.index][9].idxmin()

'express.v.02'

In [212]:
vecs_df[vecs_df["root_hypernym"] == 'express.v.02'][9].std()

0.452676883294381

In [226]:
# variances of vector attributes inside the given group
vecs_df.groupby("root_hypernym").std().loc[relevant_roots.index]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,...,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200
root_hypernym,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
act.v.01,0.498246,0.499595,0.473051,0.472069,0.489179,0.446847,0.500209,0.474015,0.468481,0.491471,0.496744,0.492766,0.495250,0.495451,0.497237,0.488564,0.500133,0.495841,0.461182,0.481458,0.499595,0.498246,0.499252,0.495648,0.499990,...,0.467949,0.499595,0.481458,0.500173,0.484510,0.500341,0.499934,0.495451,0.499990,0.498498,0.490070,0.499344,0.484872,0.486615,0.499671,0.496030,0.488874,0.499595,0.500329,0.494836,0.483399,0.486615,0.499344,0.499516,0.499157
be.v.01,0.500967,0.496991,0.500543,0.494896,0.501721,0.473831,0.498699,0.501297,0.489522,0.476168,0.492408,0.500967,0.500967,0.476168,0.487927,0.495992,0.494896,0.499409,0.489522,0.489522,0.492408,0.484429,0.486230,0.486230,0.489522,...,0.491015,0.484429,0.501533,0.498699,0.496991,0.501674,0.500024,0.500024,0.501721,0.501297,0.501674,0.492408,0.478395,0.498699,0.499409,0.501533,0.501533,0.491015,0.501721,0.500967,0.500967,0.499409,0.501674,0.498699,0.498699
change.v.01,0.485164,0.490020,0.497242,0.492636,0.495937,0.492365,0.492903,0.499718,0.499553,0.485917,0.492365,0.496138,0.490635,0.494865,0.488397,0.497242,0.482370,0.495732,0.472469,0.490635,0.500232,0.492636,0.494165,0.499464,0.492089,...,0.485543,0.499553,0.440509,0.499864,0.500378,0.500376,0.500358,0.499272,0.499638,0.494865,0.499272,0.500358,0.494636,0.492365,0.500369,0.496138,0.499464,0.489061,0.495937,0.496525,0.482785,0.498711,0.500342,0.499793,0.481526
change.v.02,0.483532,0.493521,0.496053,0.480622,0.499543,0.489539,0.495768,0.498781,0.496053,0.491645,0.449058,0.492798,0.466478,0.486706,0.491242,0.500244,0.495768,0.496053,0.485686,0.499787,0.499264,0.492038,0.481815,0.499896,0.496053,...,0.494859,0.500504,0.461416,0.500517,0.493521,0.498414,0.498781,0.495768,0.500244,0.500504,0.499896,0.500483,0.482397,0.499996,0.500244,0.479390,0.499264,0.492423,0.499996,0.497576,0.499543,0.497576,0.500309,0.500087,0.474763
connect.v.01,0.498168,0.468440,0.496461,0.496461,0.500783,0.499608,0.498168,0.501695,0.498168,0.489706,0.486897,0.453565,0.472742,0.499608,0.483802,0.486897,0.435494,0.496461,0.486897,0.492233,0.502735,0.476731,0.476731,0.483802,0.500783,...,0.492233,0.498168,0.435494,0.498168,0.499608,0.498168,0.476731,0.492233,0.498168,0.500783,0.498168,0.472742,0.486897,0.498168,0.498168,0.458861,0.492233,0.501695,0.486897,0.468440,0.502865,0.453565,0.480416,0.480416,0.468440
entity.n.01,0.494926,0.495297,0.499808,0.497095,0.498952,0.499608,0.499001,0.497830,0.499993,0.497254,0.497583,0.499526,0.498571,0.499848,0.499781,0.495822,0.498881,0.494092,0.491901,0.499833,0.495172,0.499938,0.500008,0.499734,0.499803,...,0.498330,0.499955,0.490912,0.499734,0.499933,0.500009,0.493984,0.499904,0.494253,0.497964,0.499503,0.499877,0.487481,0.498664,0.492263,0.493704,0.499685,0.496388,0.493355,0.499980,0.499938,0.499159,0.498722,0.499929,0.497305
express.v.02,0.501324,0.491565,0.463073,0.403768,0.493773,0.419948,0.477994,0.427332,0.452677,0.479804,0.491565,0.499285,0.501155,0.500238,0.500917,0.469960,0.501324,0.498706,0.463073,0.476105,0.498058,0.491565,0.501324,0.501460,0.476105,...,0.385503,0.501460,0.500917,0.498706,0.474137,0.500917,0.460606,0.501426,0.494771,0.494771,0.472089,0.483194,0.430858,0.495698,0.501426,0.472089,0.472089,0.500612,0.493773,0.501426,0.474137,0.477994,0.501324,0.494771,0.491565
get.v.01,0.486446,0.500305,0.499083,0.480130,0.497613,0.489200,0.501280,0.489200,0.499083,0.502497,0.497613,0.502740,0.489200,0.495893,0.495893,0.459335,0.468547,0.502740,0.499083,0.501280,0.486446,0.489200,0.497613,0.499083,0.499083,...,0.436995,0.501280,0.483425,0.495893,0.502497,0.489200,0.501280,0.454257,0.499083,0.480130,0.499083,0.495893,0.493919,0.423659,0.464095,0.495893,0.501280,0.480130,0.486446,0.493919,0.491689,0.493919,0.502740,0.502497,0.500305
make.v.03,0.470465,0.493276,0.497676,0.49032

In [229]:
# variances of vector attributes inside the given group
vecs_df.groupby("root_hypernym").var().loc[relevant_roots.index]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,...,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200
root_hypernym,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
act.v.01,0.248249,0.249596,0.223777,0.222849,0.239296,0.199672,0.250209,0.224690,0.219474,0.241543,0.246755,0.242818,0.245273,0.245472,0.247245,0.238695,0.250133,0.245858,0.212689,0.231802,0.249596,0.248249,0.249253,0.245667,0.249990,...,0.218976,0.249596,0.231802,0.250173,0.234750,0.250341,0.249934,0.245472,0.249990,0.248500,0.240169,0.249345,0.235101,0.236794,0.249671,0.246046,0.238997,0.249596,0.250329,0.244862,0.233674,0.236794,0.249345,0.249516,0.249157
be.v.01,0.250968,0.247000,0.250543,0.244922,0.251724,0.224516,0.248701,0.251299,0.239632,0.226736,0.242466,0.250968,0.250968,0.226736,0.238073,0.246009,0.244922,0.249410,0.239632,0.239632,0.242466,0.234672,0.236419,0.236419,0.239632,...,0.241096,0.234672,0.251535,0.248701,0.247000,0.251677,0.250024,0.250024,0.251724,0.251299,0.251677,0.242466,0.228862,0.248701,0.249410,0.251535,0.251535,0.241096,0.251724,0.250968,0.250968,0.249410,0.251677,0.248701,0.248701
change.v.01,0.235384,0.240120,0.247250,0.242691,0.245954,0.242423,0.242953,0.249718,0.249553,0.236116,0.242423,0.246153,0.240723,0.244891,0.238531,0.247250,0.232681,0.245751,0.223227,0.240723,0.250232,0.242691,0.244199,0.249464,0.242151,...,0.235752,0.249553,0.194048,0.249864,0.250378,0.250376,0.250358,0.249272,0.249638,0.244891,0.249272,0.250358,0.244665,0.242423,0.250369,0.246153,0.249464,0.239180,0.245954,0.246537,0.233081,0.248712,0.250342,0.249793,0.231867
change.v.02,0.233803,0.243563,0.246068,0.230998,0.249543,0.239649,0.245785,0.248782,0.246068,0.241715,0.201653,0.242850,0.217602,0.236882,0.241319,0.250244,0.245785,0.246068,0.235891,0.249787,0.249265,0.242102,0.232146,0.249896,0.246068,...,0.244885,0.250505,0.212904,0.250518,0.243563,0.248417,0.248782,0.245785,0.250244,0.250505,0.249896,0.250483,0.232707,0.249996,0.250244,0.229815,0.249265,0.242480,0.249996,0.247582,0.249543,0.247582,0.250309,0.250087,0.225400
connect.v.01,0.248171,0.219436,0.246473,0.246473,0.250784,0.249608,0.248171,0.251698,0.248171,0.239812,0.237069,0.205721,0.223485,0.249608,0.234065,0.237069,0.189655,0.246473,0.237069,0.242294,0.252743,0.227273,0.227273,0.234065,0.250784,...,0.242294,0.248171,0.189655,0.248171,0.249608,0.248171,0.227273,0.242294,0.248171,0.250784,0.248171,0.223485,0.237069,0.248171,0.248171,0.210554,0.242294,0.251698,0.237069,0.219436,0.252874,0.205721,0.230799,0.230799,0.219436
entity.n.01,0.244952,0.245319,0.249808,0.247104,0.248953,0.249608,0.249002,0.247835,0.249993,0.247262,0.247589,0.249526,0.248573,0.249848,0.249781,0.245840,0.248882,0.244127,0.241966,0.249833,0.245195,0.249938,0.250008,0.249735,0.249803,...,0.248332,0.249955,0.240994,0.249735,0.249933,0.250009,0.244021,0.249904,0.244286,0.247968,0.249503,0.249877,0.237638,0.248666,0.242323,0.243744,0.249685,0.246401,0.243399,0.249980,0.249938,0.249160,0.248723,0.249929,0.247313
express.v.02,0.251326,0.241636,0.214436,0.163029,0.243812,0.176357,0.228478,0.182613,0.204916,0.230212,0.241636,0.249286,0.251156,0.250238,0.250918,0.220862,0.251326,0.248708,0.214436,0.226676,0.248062,0.241636,0.251326,0.251462,0.226676,...,0.148613,0.251462,0.250918,0.248708,0.224806,0.250918,0.212158,0.251428,0.244798,0.244798,0.222868,0.233476,0.185639,0.245716,0.251428,0.222868,0.222868,0.250612,0.243812,0.251428,0.224806,0.228478,0.251326,0.244798,0.241636
get.v.01,0.236630,0.250305,0.249084,0.230525,0.247619,0.239316,0.251282,0.239316,0.249084,0.252503,0.247619,0.252747,0.239316,0.245910,0.245910,0.210989,0.219536,0.252747,0.249084,0.251282,0.236630,0.239316,0.247619,0.249084,0.249084,...,0.190965,0.251282,0.233700,0.245910,0.252503,0.239316,0.251282,0.206349,0.249084,0.230525,0.249084,0.245910,0.243956,0.179487,0.215385,0.245910,0.251282,0.230525,0.236630,0.243956,0.241758,0.243956,0.252747,0.252503,0.250305
make.v.03,0.221338,0.243321,0.247682,0.24041

In [230]:
# variances of vector attributes inside the given group
vecs_df.groupby("root_hypernym").var().loc[relevant_roots.index].describe()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,...,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200
count,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,...,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000
mean,0.241592,0.241051,0.242945,0.229574,0.242282,0.230379,0.244985,0.238591,0.239720,0.240765,0.235601,0.239290,0.236173,0.241390,0.244730,0.240538,0.232728,0.248158,0.229100,0.241170,0.248102,0.243138,0.243603,0.246834,0.243589,...,0.228582,0.246918,0.224334,0.248496,0.243826,0.249215,0.242713,0.244384,0.248372,0.247191,0.242506,0.242865,0.232963,0.241909,0.245075,0.238527,0.245855,0.243920,0.246956,0.245084,0.241676,0.239788,0.245613,0.247703,0.239321
std,0.009754,0.008774,0.010875,0.023501,0.014245,0.023098,0.008140,0.020029,0.014572,0.009274,0.019314,0.015583,0.015715,0.008603,0.005105,0.011951,0.020306,0.003484,0.015777,0.008383,0.004358,0.007149,0.009137,0.005370,0.007809,...,0.029870,0.005207,0.023362,0.003797,0.012023,0.003191,0.011934,0.011791,0.003553,0.005207,0.014471,0.011185,0.017460,0.018392,0.009343,0.012322,0.008227,0.007242,0.005127,0.008421,0.009852,0.013227,0.008909,0.005306,0.012488
min,0.221338,0.219436,0.214436,0.163029,0.199064,0.176357,0.228013,0.182613,0.204916,0.222219,0.193299,0.205721,0.203238,0.226425,0.234065,0.210989,0.189655,0.240415,0.198721,0.226676,0.236630,0.227273,0.226080,0.234065,0.226676,...,0.148613,0.234672,0.188635,0.236217,0.211783,0.239316,0.212158,0.206349,0.239094,0.230525,0.199064,0.217046,0.185639,0.179487,0.215385,0.210554,0.222868,0.229176,0.236630,0.219436,0.224806,0.205721,0.225635,0.230799,0.211770
25%,0.234198,0.238314,0.244663,0.223333,0.240425,0.224993,0.243661,0.237137,0.233361,0.236687,0.236312,0.236032,0.226943,0.234301,0.241392,0.237035,0.223782,0.245911,0.216634,0.233721,0.246591,0.239896,0.238364,0.246305,0.240261,...,0.223170,0.244360,0.209963,0.248340,0.244005,0.248489,0.244249,0.243088,0.247804,0.246747,0.243927,0.237485,0.228940,0.243246,0.243383,0.231051,0.243896,0.240847,0.244348,0.244660,0.233279,0.231300,0.246055,0.248524,0.234019
50%,0.246326,0.243442,0.246862,0.235706,0.248162,0.239482,0.248851,0.248344,0.246875,0.241629,0.242444,0.244501,0.240598,0.245691,0.246661,0.244665,0.239861,0.249059,0.234466,0.241508,0.249430,0.243090,0.247843,0.249343,0.245721,...,0.241695,0.249574,0.229303,0.249799,0.249722,0.250175,0.247263,0.249227,0.249361,0.248602,0.248628,0.249611,0.236085,0.248196,0.248438,0.244827,0.249572,0.245463,0.248336,0.247248,0.244424,0.246125,0.249595,0.249307,0.242869
75%,0.249802,0.246944,0.249288,0.246086,0.250474,0.245815,0.250374,0.249728,0.249436,0.247360,0.247612,0.250187,0.248278,0.246448,0.248182,0.249111,0.248108,0.250727,0.238991,0.249697,0.250333,0.249361,0.250119,0.249667,0.249710,...,0.247471,0.250373,0.246414,0.250496,0.250799,0.250934,0.250084,0.250311,0.250775,0.249901,0.249466,0.250294,0.244488,0.249210,0.250100,0.246855,0.250499,0.249481,0.250549,0.250138,0.249839,0.249048,0.250572,0.249895,0.249043
max,0.251326,0.250305,0.250749,0.250430,0.252577,0.251933,0.251282,0.251698,0.251933,0.252503,0.251514,0.252747,0.251156,0.250238,0.250918,0.252363,0.251326,0.252747,0.250939,0.251282,0.252743,0.251514,0.251514,0.251462,0.251933,...,0.250190,0.251462,0.251535,0.251104,0.252503,0.252363,0.251399,0.252363,0.251933,0.251299,0.251677,0.251206,0.250268,0.251289,0.251428,0.251535,0.251535,0.251698,0.252577,0.251428,0.252874,0.250495,0.252747,0.252503,0.250407


In [231]:
# variances of vector attributes inside the given group
vecs_df.groupby("root_hypernym").var().loc["think.v.03"].describe()

count    200.000000
mean       0.241103
std        0.013302
min        0.172162
25%        0.237472
50%        0.246773
75%        0.249719
max        0.250649
Name: think.v.03, dtype: float64